In [10]:
from __future__ import print_function

import sys
import os
from glob import glob
from time import time
%matplotlib inline
#%load_ext autoreload
#%autoreload 2
import matplotlib.pyplot as plt
import pandas as pd
import tables as tb
import numpy as np
import math
#plt.rcParams['figure.figsize'] = 10,8
import datetime

from   invisible_cities.database import load_db
from   invisible_cities.core.system_of_units_c import SystemOfUnits
import invisible_cities.sierpe.blr as blr
import invisible_cities.core.mpl_functions as mpl
import invisible_cities.reco.wfm_functions as wfm
import invisible_cities.reco.tbl_functions as tbl
import invisible_cities.core.peak_functions_c as cpf
import invisible_cities.reco.pmaps_functions as pf
import invisible_cities.core.sensor_functions as sf
from   invisible_cities.core.core_functions import define_window

import invisible_cities.core.pmaps_functions_c as cpm
from   invisible_cities.core.core_functions import lrange
 
units = SystemOfUnits()
t0 = time()

In [11]:
import S1S2prop as prop
import plotting as plot

In [22]:
#%%
mydf_file = os.environ['IC_DATA']+'/pmaps_JM/Kr/pmaps_NEXT_v0_08_09_Kr_ACTIVE_1_0_7bar__10000.root.h5'
print(mydf_file)

S1df, S2df, S2Sidf = pf.read_pmaps(mydf_file)

#mydf
#mydf = pf.read_pmaps(mydf_file)
#list(map(type, mydf))
#S1df   = mydf[0]
#S2df   = mydf[1]
#S2Sidf = mydf[2]

print('S1df entries (tbins x events):',len(S1df))
print('S2df entries (tbins x events):',len(S2df))
print('S2Sidf entries:',len(S2Sidf))
type(S1df)
print('Keys of S1df panda dataframe: {} '  .format(S1df.keys()))
print('Keys of S2df panda dataframe: {} '  .format(S2df.keys()))
print('Keys of S2Sidf panda dataframe: {} '.format(S2Sidf.keys()))

# Convert S12df object  (an S12 pytable read as a PD dataframe) and return an S12L dictionary (list of dict, first dict)
S1dict   = pf.df_to_pmaps_dict(S1df,10000)  # revisar NO poner event max
S2dict   = pf.df_to_pmaps_dict(S2df,10000)
S2Sidict = pf.df_to_s2si_dict (S2Sidf,10000)

evid_S1min   = sorted(S1dict.keys())  [0]
evid_S1max   = sorted(S1dict.keys())  [-1]
evid_S2min   = sorted(S2dict.keys())  [0]
evid_S2max   = sorted(S2dict.keys())  [-1]
evid_S2Simin = sorted(S2Sidict.keys())[0]
evid_S2Simax = sorted(S2Sidict.keys())[-1]

print('First/last event ID (first item in sorted S1 dictionary):   {}/{}'.format(evid_S1min,     evid_S1max))
print('First/last event ID (first item in sorted S2 dictionary):   {}/{}'.format(evid_S2min,     evid_S2max))
print('First/last event ID (first item in sorted S2Si dictionary): {}/{}'.format(evid_S2Simin, evid_S2Simax))
print('Total number of events in S1   = {}'.format(len(S1dict)))
print('Total number of events in S2   = {}'.format(len(S2dict)))
print('Total number of events in S2Si = {}'.format(len(S2Sidict)))
len(S1dict), type(S1dict), len(S2dict), type(S2dict), len(S2Sidict), type(S2Sidict)



/Users/neus/InvCities/data/pmaps_JM/Kr/pmaps_NEXT_v0_08_09_Kr_ACTIVE_1_0_7bar__10000.root.h5
S1df entries (tbins x events): 51993
S2df entries (tbins x events): 91086
S2Sidf entries: 45115
Keys of S1df panda dataframe: Index(['event', 'evtDaq', 'peak', 'time', 'ene'], dtype='object') 
Keys of S2df panda dataframe: Index(['event', 'evtDaq', 'peak', 'time', 'ene'], dtype='object') 
Keys of S2Sidf panda dataframe: Index(['event', 'evtDaq', 'peak', 'nsipm', 'nsample', 'ene'], dtype='object') 
First/last event ID (first item in sorted S1 dictionary):   0/9999
First/last event ID (first item in sorted S2 dictionary):   0/9999
First/last event ID (first item in sorted S2Si dictionary): 0/9998
Total number of events in S1   = 6980
Total number of events in S2   = 9977
Total number of events in S2Si = 8514


(6980, dict, 9977, dict, 8514, dict)

In [66]:
import invisible_cities.database.load_db as DB
DataPMT = DB.DataPMT()
DataSiPM = DB.DataSiPM()

pi = np.pi

In [67]:
#S2Sidict

In [68]:
#S2Sidf

In [69]:
### %%
class S12Prop:
    """
    input: S1L
    returns a S1F object for specific peak
    """        
    
    def __init__(self, S12dict):
        self.S12dict = S12dict
        self.length  = len(self.S12dict)
        self.prop()

    def dict(self):
            return self.S12dict

#    def length(self):
#        return len(self.S12dict)

#test len arrays is == length


    def prop(self):
        self.IDX        = np.zeros(self.length, dtype=np.int) # array index for emax
        self.tIDX       = np.zeros(self.length, dtype=np.double) # time value for IDX
        self.tmin       = np.zeros(self.length, dtype=np.double) # minimum t value
        self.tmax       = np.zeros(self.length, dtype=np.double) # maximum t value
        self.twidth     = np.zeros(self.length, dtype=np.double) # t width of signal (mus) 
        self.tmean      = np.zeros(self.length, dtype=np.double) # mean time
        self.emin       = np.zeros(self.length, dtype=np.double) # min energy the S1/S2 pulse
        self.emax       = np.zeros(self.length, dtype=np.double) # max energy the S1/S2 pulse
        self.etot       = np.zeros(self.length, dtype=np.double) # total energy
        self.emean      = np.zeros(self.length, dtype=np.double) # total energy
        

        # lists to be filled in the loop. Lists name added 'l': lVARIABLE
        lIDX     = []
        ltIDX    = []
        lemax    = []
        ltmin    = []
        ltmax    = []
        ltmean   = []
        ltwidth  = []
        lemin    = []
        lemax    = []
        letot    = []
        lemean   = []

        # 1st loop over dic of events
        # 2nd loop over dic of peaks,(ts,Es)=namedTuple (can also be accessed via value.t, value.E)
        for evtID, evt in self.S12dict.items():  
            for peakID, (ts, Es) in evt.items():  
                if(evtID     == 9999): break  #------> PATCH
                if(peakID        == 1):   break  #------> select one peak 

                lIDX       .append(np.argmax(Es))
                ltIDX      .append(ts[lIDX[-1]]                / units.mus) # use last value of lIDX: [-1]
                ltmin      .append(np.amin  (ts)               / units.mus)
                ltmax      .append(np.amax  (ts)               / units.mus)
                ltwidth    .append((np.amax (ts) - np.amin(ts))/ units.mus)
                ltmean     .append(np.mean  (ts)               / units.mus)
                lemin      .append(np.amin  (Es))
                lemax      .append(np.amax  (Es))
                letot      .append(np.sum   (Es))
                lemean     .append(np.mean  (Es))
                
                
        # convert lists to numpy arrays
        self.IDX      = np.array(lIDX)  
        self.tIDX     = np.array(ltIDX)
        self.tmin     = np.array(ltmin)
        self.tmax     = np.array(ltmax)
        self.twidth   = np.array(ltwidth)
        self.tmean    = np.array(ltmean)
        self.emax     = np.array(lemax)
        self.emin     = np.array(lemin) 
        self.etot     = np.array(letot)
        self.emean    = np.array(lemean)

         
    def S1S2mapd(self, other):
        filt_dict = lambda x, y: dict([ (i , x[i] ) for i in x if i in set(y) ])
      
        keys_S1 = set(self.S12dict.keys())
        keys_S2 = set(other.S12dict.keys())
        intsect = keys_S1 & keys_S2

        S1map = filt_dict(self.S12dict,intsect)
        S2map = filt_dict(other.S12dict,intsect)
        return S12Prop(S1map), S12Prop(S2map)
        




In [70]:
S1 = S12Prop(S1dict)
S2 = S12Prop(S2dict)

In [219]:
from collections import namedtuple

def fsipms_tuple(S2Sidict):
    sipms_tuple = []   
    for evtID, evt in S2Sidict.items():  
       # if evtID != 0: break 
        for peakID, sipms in evt.items(): 
            if(peakID >=1):   break
            for sipmID, Es in sipms.items(): 
                print(sipmID) 
                sipms_tuple.append((sipmID,np.sum(Es)))           
    return sipms_tuple
    

In [220]:
sipms_tuple = fsipms_tuple(S2Sidict)

1515
1069
292
284
285
293
1163
1413
921
929
105
98
106
371
372
363
364
1602
1603
690
698
110
111
1751
1743
561
554
562
584
585
592
1538
1546
975
967
99
100
684
1001
58
59
537
538
546
1549
1550
209
210
218
217
693
694
528
536
1220
796
797
1543
1072
1143
1135
545
1494
718
719
405
406
120
121
1674
632
633
831
1516
1517
1525
1435
1436
585
839
110
102
814
815
410
418
250
582
887
879
579
122
320
321
1199
1541
1687
693
189
182
190
1381
1382
1238
116
262
263
285
1172
1165
1173
800
808
879
646
647
1223
51
52
294
285
286
1349
1350
952
953
1284
853
401
409
954
662
654
808
879
404
395
396
1485
684
685
1106
1114
1739
811
803
1493
1071
615
614
607
1365
224
225
1488
1496
1095
771
511
740
738
344
345
352
353
952
953
1284
1100
1099
1091
1092
763
1564
1556
1557
387
388
395
186
1001
1009
656
727
1131
1132
1394
1402
1214
1686
1678
502
494
503
780
1558
1550
785
786
794
1186
708
999
991
1489
1490
1004
1388
1380
1381
1389
521
522
709
281
1121
1129
1419
1427
1133
1134
1428
1420
707
944
952
1023
1139
681
689
1

In [221]:
sipms_tuple

[(1515, 57.934568405151367),
 (1069, 41.890523910522461),
 (292, 41.947311401367188),
 (284, 103.64170837402344),
 (285, 128.83237075805664),
 (293, 56.922595977783203),
 (1163, 83.838634490966797),
 (1413, 321.07379150390625),
 (921, 74.994647979736328),
 (929, 40.540552139282227),
 (105, 69.475906372070312),
 (98, 43.349363327026367),
 (106, 90.343667984008789),
 (371, 108.4232063293457),
 (372, 71.676338195800781),
 (363, 56.77568244934082),
 (364, 64.629348754882812),
 (1602, 130.3422679901123),
 (1603, 86.443586349487305),
 (690, 316.49636077880859),
 (698, 68.439060211181641),
 (110, 44.023124694824219),
 (111, 72.202150344848633),
 (1751, 68.788726806640625),
 (1743, 64.349292755126953),
 (561, 60.117185592651367),
 (554, 71.395221710205078),
 (562, 177.29400062561035),
 (584, 142.92697525024414),
 (585, 110.5728874206543),
 (592, 43.775970458984375),
 (1538, 47.866222381591797),
 (1546, 41.089155197143555),
 (975, 112.73305511474609),
 (967, 77.031572341918945),
 (99, 142.15964

In [216]:
S2Sidict

{0: {0: {1515: array([  0.        ,   0.        ,   0.        ,   0.        ,
           30.98270416,  26.95186424])}},
 1: {0: {1069: array([  0.        ,   0.        ,   0.        ,   0.        ,
           19.9916153 ,  21.89890862])}},
 2: {0: {284: array([  0.        ,  44.91878128,  58.72292709]),
   285: array([  0.        ,  68.91503143,  59.91733932]),
   292: array([  0.        ,  21.10059547,  20.84671593]),
   293: array([  0.        ,  28.0588932 ,  28.86370277])}},
 3: {0: {1163: array([  0.        ,   0.        ,   0.        ,   0.        ,
           27.66733551,  24.1921196 ,  31.97917938])}},
 4: {0: {1413: array([  76.50045776,  152.74253845,   91.83079529])}},
 5: {0: {921: array([  0.        ,   0.        ,  25.17119789,  27.96054459,  21.8629055 ]),
   929: array([  0.        ,   0.        ,  20.52487373,  20.01567841])}},
 6: {0: {98: array([  0.        ,   0.        ,  20.98043823,   0.        ,  22.36892509]),
   105: array([  0.        ,   0.        ,  25.8538

In [138]:
b==78.63371467590332

False

In [139]:
b

57.934568405151367

In [112]:
S2Sidict

{0: {0: {1515: array([  0.        ,   0.        ,   0.        ,   0.        ,
           30.98270416,  26.95186424])}},
 1: {0: {1069: array([  0.        ,   0.        ,   0.        ,   0.        ,
           19.9916153 ,  21.89890862])}},
 2: {0: {284: array([  0.        ,  44.91878128,  58.72292709]),
   285: array([  0.        ,  68.91503143,  59.91733932]),
   292: array([  0.        ,  21.10059547,  20.84671593]),
   293: array([  0.        ,  28.0588932 ,  28.86370277])}},
 3: {0: {1163: array([  0.        ,   0.        ,   0.        ,   0.        ,
           27.66733551,  24.1921196 ,  31.97917938])}},
 4: {0: {1413: array([  76.50045776,  152.74253845,   91.83079529])}},
 5: {0: {921: array([  0.        ,   0.        ,  25.17119789,  27.96054459,  21.8629055 ]),
   929: array([  0.        ,   0.        ,  20.52487373,  20.01567841])}},
 6: {0: {98: array([  0.        ,   0.        ,  20.98043823,   0.        ,  22.36892509]),
   105: array([  0.        ,   0.        ,  25.8538

In [72]:
for evtID, evt in S2dict.items():  
    if evtID != 0: break 
    for peakID, (ts, Es) in evt.items(): 
        print(ts)
            

[ 500689.375    501737.5      502737.5      503737.5      504737.5
  505737.5      506737.5      507737.5      508737.5      509737.5
  510729.28125]


In [87]:
S2Sidict[0][0]

{1515: array([  0.        ,   0.        ,   0.        ,   0.        ,
         30.98270416,  26.95186424])}

In [61]:
#S2dict

In [ ]:
def integrate_charge(d):
    newd = dict((key, np.sum(value)) for key, value in d.items())
    return map(np.array, list(zip(*newd.items())))
